# Get weather for next 12 hours

In [1]:
import pandas as pd
import datetime as dt

import json
import requests
import os

#from utils import weather

API_KEY = "9TS2B3LDPRU8UYW7QBPLEUV6S"

REGIONS_DICTIONARY_FILE = "data/0_raw_other_data/regions.csv"
OUTPUT_FOLDER = "data/1_weather_for_12_hours"

### Read regions data

In [2]:
df_regions = pd.read_csv(REGIONS_DICTIONARY_FILE)
df_regions.head(10)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5
5,Житомирська,Житомир,Zhytomyr,Житомирщина,6
6,Закарпатська,Ужгород,Uzhgorod,Закарпаття,7
7,Запорізька,Запоріжжя,Zaporozhye,Запоріжжя,8
8,Івано-Франківська,Івано-Франківськ,Ivano-Frankivsk,Івано-Франківщина,9
9,Київська,Київ,Kyiv,Київщина,10


### Set variables for API integration

In [3]:
region = "Донеччина"
today_date = "2023-03-09"
level = "hours"
country = "Ukraine"
start_date = end_date = today_date

city = df_regions[df_regions["region_alt"] == region]["center_city_en"].values[0]
location = f"{city},{country}"

file_name = f"weather_{city.lower()}_{start_date}_{end_date}.json"

### Get weather json

In [5]:
if not os.path.isfile(f"{OUTPUT_FOLDER}/{file_name}"):
    #city_weather_json = weather.get_weather(location, start_date, end_date, level)
    url_base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    url_key = f"?key={API_KEY}"
    url_location = location
    url_start_data = start_date
    url_end_data = end_date
    url_level = level

    url = f"{url_base_url}/{url_location}/{url_start_data}/{url_end_data}/{url_key}"

    response = requests.request("GET", url)
    city_weather_json = response.json()
    
    with open(f"{OUTPUT_FOLDER}/{file_name}", 'w') as outfile:
        json.dump(city_weather_json, outfile)
        #outfile.write(city_weather_json)
        
else:
    print(f"Weather data for the \nregion {region}; \nstart_date {start_date}; \nend_date {end_date}; \nis ready")

Weather data for the 
region Донеччина; 
start_date 2023-03-09; 
end_date 2023-03-09; 
is ready


### Create hours DataFrame

In [6]:
with open(f"{OUTPUT_FOLDER}/{file_name}") as outfile:
    weather_json = json.load(outfile)

hours_weather_json = []

for day in weather_json['days']:
    for hour in day['hours']:
        hours_weather_json.append(hour)

hours_df = pd.DataFrame(hours_weather_json)

hours_df.head(10)

,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,pressure,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source
0,00:00:00,1678312800,42.2,38.6,92.63,40.2,0.0,0.0,0.0,0.0,...,1009.0,None,76.5,None,None,None,Partially cloudy,partly-cloudy-night,[remote],obs
1,01:00:00,1678316400,41.6,39.4,93.92,40.0,0.0,0.0,0.0,0.0,...,1009.0,None,74.0,None,None,None,Partially cloudy,partly-cloudy-night,[remote],obs
2,02:00:00,1678320000,41.4,41.4,93.92,39.8,0.0,0.0,0.0,0.0,...,1009.0,None,100.0,None,None,None,Overcast,cloudy,[remote],obs
3,03:00:00,1678323600,41.6,40.1,95.24,40.4,0.0,0.0,0.0,0.0,...,1009.0,None,96.5,None,None,None,Overcast,cloudy,[remote],obs
4,04:00:00,1678327200,41.8,41.8,95.91,40.7,0.0,0.0,0.0,0.0,...,1009.0,None,100.0,None,None,None,Overcast,cloudy,[remote],obs
5,05:00:00,1678330800,41.6,39.6,97.26,40.9,0.0,0.0,0.0,0.0,...,1009.0,None,97.9,None,None,None,Overcast,cloudy,[remote],obs
6,06:00:00,1678334400,42.0,39.2,97.26,41.3,0.0,0.0,0.0,0.0,...,1008.0,None,91.4,None,None,None,Overcast,cloudy,[remote],obs
7,07:00:00,1678338000,42.3,38.7,97.94,41.8,0.0,0.0,0.0,0.0,...,1008.0,None,93.9,None,None,None,Overcast,cloudy,[remote],obs
8,08:00:00,1678341600,44.7,39.5,91.44,42.3,0.0,0.0,0.0,0.0,...,1008.0,None,98.8,None,None,None,Overcast,cloudy,[remote],obs
9,09:00:00,1678345200,46.7,42.1,83.66,42.0,0.0,0.0,0.0,0.0,...,1008.0,None,97.6,None,None,None,Overcast,cloudy,[remote],obs
